In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore') 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [79]:
file_version = 35
CFG = {
    'EPOCHS': 30,
    'LEARNING_RATE':1e-2,    # 0.01
    'BATCH_SIZE':256,  #256
    'SEED':43,
    'log_transform_turn': 1,  # 로그 트랜스폼을 몇번 돌릴 것인가?
    'skew_cut':0.01,           # 왜도 얼마 이상을 로그값으로 정규화할 것인가?(100이면 안하겠다는 것)
    'vif_cut': 10,           # 다중공선성 vif 얼마 이상을 제거할 것인가?
    'corr_cut': 0.006,     # 일정 상관관계 이하 칼럼은 드랍
    'outlier_corr_cut': 100,    # 이값 이상의 상관관계를 갖는 칼럼에 대해서만 아웃라이어 제거(100이면 미적용)
    'T_Thresh': 0.35,     # Teacher model train Threshhold 최초 0.35
    'S_Thresh': 0.35,     # Student model train Threshold 최초 0.35
    'reduct' : 'sum',      # sum, mean
    'drop_rate': 0.2,
    
}

In [81]:
import pickle
#불러오기
with open(f"./data/{file_version}_refined_train_X", 'rb') as pickle_filename:
	train_X = pickle.load(pickle_filename)

with open(f"./data/{file_version}_refined_train_y", 'rb') as pickle_filename:
	train_y = pickle.load(pickle_filename)
    
with open(f"./data/{file_version}_refined_val_X", 'rb') as pickle_filename:
	val_X = pickle.load(pickle_filename)

with open(f"./data/{file_version}_refined_val_y", 'rb') as pickle_filename:
	val_y = pickle.load(pickle_filename)     

with open(f"./data/{file_version}_refined_test", 'rb') as pickle_filename:
	test = pickle.load(pickle_filename)

In [82]:
train_y.value_counts()

0.0    10314
1.0      962
Name: Y_LABEL, dtype: int64

In [93]:
train_X.shape

(11276, 40)

In [83]:
test_stage_features = test.columns
test_stage_features

Index(['encoder__x0_COMPONENT2', 'encoder__x0_COMPONENT3',
       'encoder__x0_COMPONENT4', 'encoder__x1_y2', 'encoder__x1_y3',
       'encoder__x1_y4', 'encoder__x1_y5', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [91]:
train_y

3283     0.0
9759     0.0
174      0.0
12815    0.0
14017    0.0
        ... 
12579    0.0
12507    0.0
10069    0.0
13857    0.0
4433     0.0
Name: Y_LABEL, Length: 11276, dtype: float64

In [84]:
len(test_stage_features)

23

In [85]:
class CustomDataset(Dataset):
    def __init__(self, data_X, data_y, distillation=False):
        super(CustomDataset, self).__init__()
        self.data_X = data_X
        self.data_y = data_y
        self.distillation = distillation
        
    def __len__(self):
        return len(self.data_X)
    
    def __getitem__(self, index):
        if self.distillation:
            # 지식 증류 학습 시
            teacher_X = torch.Tensor(self.data_X.iloc[index])
            student_X = torch.Tensor(self.data_X[test_stage_features].iloc[index])
            y = self.data_y.values[index]
            return teacher_X, student_X, y
        else:
            if self.data_y is None:
                test_X = torch.Tensor(self.data_X.iloc[index])
                return test_X
            else:
                teacher_X = torch.Tensor(self.data_X.iloc[index])
                y = self.data_y.values[index]
                return teacher_X, y

In [86]:
train_dataset = CustomDataset(train_X, train_y, False)
val_dataset = CustomDataset(val_X, val_y, False)
list(train_dataset)[:1]

[(tensor([ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
           1.0000, -1.0430,  1.0329, -0.1537, -0.4862, -1.1424, -0.4931, -0.0544,
           0.9904, -0.1017,  5.6677, -0.0957,  2.2429,  0.6061, -0.0969, -0.3229,
          -0.2594, -0.2194,  0.4350, -0.3730, -0.4347,  0.8037,  0.2552, -0.6452,
          -2.0525, -0.4578, -0.8064,  1.2249, -0.2640, -0.1545, -1.9767,  0.5502]),
  0.0)]

In [92]:
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
list(train_loader)[:1] 

[[tensor([[ 0.0000,  1.0000,  0.0000,  ..., -0.1545, -1.0558,  0.8471],
          [ 0.0000,  0.0000,  1.0000,  ..., -0.1545, -1.2658,  0.9663],
          [ 0.0000,  1.0000,  0.0000,  ..., -0.1545,  0.2399,  0.8738],
          ...,
          [ 0.0000,  0.0000,  1.0000,  ..., -0.1545, -0.6378,  0.8776],
          [ 0.0000,  1.0000,  0.0000,  ..., -0.1545,  0.5086, -0.6609],
          [ 0.0000,  1.0000,  0.0000,  ..., -0.1545,  0.9170, -1.7116]]),
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [50]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=train_col_num, out_features=256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1024),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            nn.Linear(in_features=1024, out_features=256),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(in_features=256, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [51]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)

    best_score = 0
    best_model = None
    
    # Loss Function 정의
    criterion = nn.BCELoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss
#     criterion = nn.BCEWithLogitsLoss(reduction = CFG['reduct']).to(device)  #BCEWithLogitsLoss

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
  
        model.train()
        for X, y in tqdm(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()
            
            y_pred = model(X)
            
            loss = criterion(y_pred, y.reshape(-1, 1))  # loss_fn(input, target)
            loss.backward()
            
            optimizer.step()

            train_loss.append(loss.item())

        val_loss, val_score = validation_teacher(model, val_loader, criterion, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
    print(f"Teacher Train Best Score는 {best_score} 입니다.")
    return best_model 

In [52]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")


def validation_teacher(model, val_loader, criterion, device):
    model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = CFG['T_Thresh']     
    
    with torch.no_grad():
        for X, y in tqdm(val_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(X.to(device))
            
            loss = criterion(model_pred, y.reshape(-1, 1))
            val_loss.append(loss.item())      
            
            model_pred = model_pred.squeeze(1).to('cpu')  # squeeze함수는 차원이 1인 차원을 제거해준다.
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        #print(pred_labels[0:10])
        # 오일상태는 0: 정상, 1: 이상
        # 어느 임계점 thrshold보다 큰 예측 라벨을 1로 하고, 나머지를 0으로 할 것인가?
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        #print(pred_labels[0:10])
        cf = confusion_matrix(true_labels, pred_labels)
        val_f1 = competition_metric(true_labels, pred_labels)
        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
    return val_loss, val_f1

In [53]:
train_col_num = len(train_X.columns)
test_col_num = len(test.columns)
print(train_col_num, test_col_num) # Drop후 칼럼 갯수 (nn 시작 노드 갯수에 맞춰주기 위해 변수에 담아놓음)

40 23


In [54]:
train_X.columns

Index(['encoder__x0_COMPONENT2', 'encoder__x0_COMPONENT3',
       'encoder__x0_COMPONENT4', 'encoder__x1_y2', 'encoder__x1_y3',
       'encoder__x1_y4', 'encoder__x1_y5', 'encoder__x2_s2', 'encoder__x2_s3',
       'ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'AL', 'B', 'BA', 'BE', 'CA', 'CD',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'K', 'LI', 'MG', 'MN', 'MO', 'NA', 'NI',
       'PB', 'PQINDEX', 'S', 'SB', 'SI', 'SN', 'TI', 'V', 'V40', 'ZN'],
      dtype='object')

In [55]:
test_stage_features = test.columns
test_stage_features

Index(['encoder__x0_COMPONENT2', 'encoder__x0_COMPONENT3',
       'encoder__x0_COMPONENT4', 'encoder__x1_y2', 'encoder__x1_y3',
       'encoder__x1_y4', 'encoder__x1_y5', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [56]:
test_col_num

23

In [57]:
model = Teacher()
model.eval()
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)
pred_labels = []
true_labels = []

teacher_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

[[2520   58]
 [ 116  125]]
              precision    recall  f1-score   support

       0: 정상       0.96      0.98      0.97      2578
       1: 이상       0.68      0.52      0.59       241

    accuracy                           0.94      2819
   macro avg       0.82      0.75      0.78      2819
weighted avg       0.93      0.94      0.93      2819

Epoch [0], Train Loss : [58.55678] Val Loss : [42.85490] Val F1 Score : [0.77813]


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

[[2546   32]
 [ 127  114]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.99      0.97      2578
       1: 이상       0.78      0.47      0.59       241

    accuracy                           0.94      2819
   macro avg       0.87      0.73      0.78      2819
weighted avg       0.94      0.94      0.94      2819

Epoch [28], Train Loss : [23.83451] Val Loss : [48.10267] Val F1 Score : [0.77943]


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

[[2543   35]
 [ 123  118]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.99      0.97      2578
       1: 이상       0.77      0.49      0.60       241

    accuracy                           0.94      2819
   macro avg       0.86      0.74      0.78      2819
weighted avg       0.94      0.94      0.94      2819

Epoch [29], Train Loss : [24.16900] Val Loss : [47.24068] Val F1 Score : [0.78443]
Epoch 00030: reducing learning rate of group 0 to 1.2207e-06.
Teacher Train Best Score는 0.8071780634518277 입니다.


In [58]:
PATH1 = f"./models/{file_version}_teacher_net.pth"
torch.save(teacher_model.state_dict(), PATH1)

In [59]:
teacher_model = Teacher()
teacher_model.load_state_dict(torch.load(PATH1))
teacher_model

Teacher(
  (classifier): Sequential(
    (0): Linear(in_features=40, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=1024, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=1024, out_features=256, bias=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=256, out_features=1, bias=True)
    (13): Sigmoid()
  )
)

In [60]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=test_col_num, out_features=128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=512),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.Dropout(p=CFG['drop_rate']),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        output = self.classifier(x)
        return output

In [61]:
# distillation Function 정의

def distillation(student_logits, labels, teacher_logits, alpha, T):
    
    student_loss = nn.BCELoss(reduction = CFG['reduct'])(student_logits, labels.reshape(-1, 1)) #BCEWithLogitsLoss
#     student_loss = nn.BCEWithLogitsLoss(reduction = CFG['reduct'])(student_logits, labels.reshape(-1, 1)) #BCEWithLogitsLoss

    distillation_loss = nn.KLDivLoss(reduction='batchmean')(F.log_softmax(student_logits/T, dim=1), F.softmax(teacher_logits/T, dim=1)) * (T * T)
    
    return alpha * student_loss + (1-alpha) * distillation_loss

def distill_loss(output, target, teacher_output, loss_fn=distillation, opt=optimizer):
    loss_b = loss_fn(output, target, teacher_output, alpha=0.1, T=10)

    if opt is not None:
        opt.zero_grad()
        loss_b.backward()
        opt.step()

    return loss_b.item()

In [62]:
def student_train(s_model, t_model, optimizer, train_loader, val_loader, scheduler, device):
    s_model.to(device)
    t_model.to(device)
    
    best_score = 0
    best_model = None

    for epoch in range(CFG["EPOCHS"]):
        train_loss = []
        s_model.train()
        t_model.eval()    # teather는 이번에는 학습하지 않으므로 .eval()로 불러온다
        
        for X_t, X_s, y in tqdm(train_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()

            output = s_model(X_s)
            with torch.no_grad():
                teacher_output = t_model(X_t)
                
            loss_b = distill_loss(output, y, teacher_output, loss_fn=distillation, opt=optimizer)

            train_loss.append(loss_b)

        val_loss, val_score = validation_student(s_model, t_model, val_loader, distill_loss, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_score)
            
        if best_score < val_score:
            best_model = s_model
            best_score = val_score
    
    print(f"Student Train Best Score는 {best_score} 입니다.")
        
    return best_model

In [63]:
def validation_student(s_model, t_model, val_loader, criterion, device):
    s_model.eval()
    t_model.eval()

    val_loss = []
    pred_labels = []
    true_labels = []
    threshold = CFG['S_Thresh']
    
    with torch.no_grad():
        for X_t, X_s, y in tqdm(val_loader):
            X_t = X_t.float().to(device)
            X_s = X_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = s_model(X_s)
            teacher_output = t_model(X_t)
            
            loss_b = distill_loss(model_pred, y, teacher_output, loss_fn=distillation, opt=None)
            val_loss.append(loss_b)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
        val_f1 = competition_metric(true_labels, pred_labels)
        cf = confusion_matrix(true_labels, pred_labels)        
        print(cf)
        print(classification_report(true_labels, pred_labels, target_names=["0: 정상", "1: 이상"]))
        
    return val_loss, val_f1    

In [64]:
train_dataset = CustomDataset(train_X, train_y, True)
val_dataset = CustomDataset(val_X, val_y, True)

train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [65]:
test_stage_features

Index(['encoder__x0_COMPONENT2', 'encoder__x0_COMPONENT3',
       'encoder__x0_COMPONENT4', 'encoder__x1_y2', 'encoder__x1_y3',
       'encoder__x1_y4', 'encoder__x1_y5', 'ANONYMOUS_1', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [66]:
student_model = Student()
student_model.eval()
optimizer = torch.optim.Adam(student_model.parameters(), lr=CFG['LEARNING_RATE'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1, threshold_mode='abs',min_lr=1e-8, verbose=True)

best_student_model = student_train(student_model, teacher_model, optimizer, train_loader, val_loader, scheduler, device)


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

[[2540   38]
 [ 225   16]]
              precision    recall  f1-score   support

       0: 정상       0.92      0.99      0.95      2578
       1: 이상       0.30      0.07      0.11       241

    accuracy                           0.91      2819
   macro avg       0.61      0.53      0.53      2819
weighted avg       0.87      0.91      0.88      2819

Epoch [0], Train Loss : [8.12185] Val Loss : [7.16114] Val F1 Score : [0.52963]


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

...

  0%|          | 0/12 [00:00<?, ?it/s]

[[2551   27]
 [ 228   13]]
              precision    recall  f1-score   support

       0: 정상       0.92      0.99      0.95      2578
       1: 이상       0.33      0.05      0.09       241

    accuracy                           0.91      2819
   macro avg       0.62      0.52      0.52      2819
weighted avg       0.87      0.91      0.88      2819

Epoch [28], Train Loss : [6.53472] Val Loss : [6.70891] Val F1 Score : [0.52246]


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

[[2551   27]
 [ 230   11]]
              precision    recall  f1-score   support

       0: 정상       0.92      0.99      0.95      2578
       1: 이상       0.29      0.05      0.08       241

    accuracy                           0.91      2819
   macro avg       0.60      0.52      0.52      2819
weighted avg       0.86      0.91      0.88      2819

Epoch [29], Train Loss : [6.51987] Val Loss : [6.70148] Val F1 Score : [0.51545]
Epoch 00030: reducing learning rate of group 0 to 6.1035e-07.
Student Train Best Score는 0.5620487935862518 입니다.


In [67]:
PATH1 = f"./models/{file_version}_student_net.pth"
torch.save(best_student_model.state_dict(), PATH1)

In [68]:
best_student_model = Student()
best_student_model.load_state_dict(torch.load(PATH1))
best_student_model

Student(
  (classifier): Sequential(
    (0): Linear(in_features=23, out_features=128, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=128, out_features=512, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.01)
    (8): Linear(in_features=512, out_features=128, bias=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=128, out_features=1, bias=True)
    (13): Sigmoid()
  )
)

In [69]:
def choose_threshold(model, val_loader, device):
    model.to(device)
    model.eval()
    
#     thresholds = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    thresholds = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

    pred_labels = []
    true_labels = []
    
    best_score = 0
    best_thr = None
    with torch.no_grad():
        for _, x_s, y in tqdm(iter(val_loader)):
            x_s = x_s.float().to(device)
            y = y.float().to(device)
            
            model_pred = model(x_s)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            pred_labels += model_pred.tolist()
            true_labels += y.tolist()
        
        for threshold in thresholds:
            pred_labels_thr = np.where(np.array(pred_labels) > threshold, 1, 0)
            score_thr = competition_metric(true_labels, pred_labels_thr)
            cf1 = confusion_matrix(true_labels, pred_labels_thr)  

            print(cf1)
            print(classification_report(true_labels, pred_labels_thr, target_names=["0: 정상", "1: 이상"]))

            if best_score < score_thr:
                best_score = score_thr
                best_thr = threshold
                
    return best_thr, best_score

In [70]:
choose_threshold(best_student_model, val_loader, device)
best_threshold, best_score = choose_threshold(best_student_model, val_loader, device)
choose_threshold_result = f'Best Threshold : [{best_threshold}], Score : [{best_score:.5f}]'
print(choose_threshold_result)

  0%|          | 0/12 [00:00<?, ?it/s]

[[ 978 1600]
 [  49  192]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.38      0.54      2578
       1: 이상       0.11      0.80      0.19       241

    accuracy                           0.42      2819
   macro avg       0.53      0.59      0.37      2819
weighted avg       0.88      0.42      0.51      2819

[[1688  890]
 [ 100  141]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.65      0.77      2578
       1: 이상       0.14      0.59      0.22       241

    accuracy                           0.65      2819
   macro avg       0.54      0.62      0.50      2819
weighted avg       0.88      0.65      0.73      2819

[[2071  507]
 [ 136  105]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.80      0.87      2578
       1: 이상       0.17      0.44      0.25       241

    accuracy                           0.77      2819
   macro avg       0.55      0.62      0.56    

  0%|          | 0/12 [00:00<?, ?it/s]

[[ 978 1600]
 [  49  192]]
              precision    recall  f1-score   support

       0: 정상       0.95      0.38      0.54      2578
       1: 이상       0.11      0.80      0.19       241

    accuracy                           0.42      2819
   macro avg       0.53      0.59      0.37      2819
weighted avg       0.88      0.42      0.51      2819

[[1688  890]
 [ 100  141]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.65      0.77      2578
       1: 이상       0.14      0.59      0.22       241

    accuracy                           0.65      2819
   macro avg       0.54      0.62      0.50      2819
weighted avg       0.88      0.65      0.73      2819

[[2071  507]
 [ 136  105]]
              precision    recall  f1-score   support

       0: 정상       0.94      0.80      0.87      2578
       1: 이상       0.17      0.44      0.25       241

    accuracy                           0.77      2819
   macro avg       0.55      0.62      0.56    

In [71]:
test_datasets = CustomDataset(test, None, False)
test_loaders = DataLoader(test_datasets, batch_size = CFG['BATCH_SIZE'], shuffle=False)

In [72]:
def inference(model, test_loader, threshold, device):
    model.to(device)
    model.eval()
    # .eval함수는 evaluation 과정에서 사용하지 말아야 하는 layer들을 알아서 off 시키는 함수
    
    test_predict = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.float().to(device)
            model_pred = model(x)

            model_pred = model_pred.squeeze(1).to('cpu')
            test_predict += model_pred
        
    test_predict = np.where(np.array(test_predict) > threshold, 1, 0)  
    # threshhold 보다 큰 값을 찾아서 1로 바꾸고, 아닌 것은 0으로 변경해라
    print('Done.')
    return test_predict

In [73]:
preds = inference(best_student_model, test_loaders, best_threshold, device)

  0%|          | 0/24 [00:00<?, ?it/s]

Done.


In [74]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['Y_LABEL'] = preds
submit.head()

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,1
4,TEST_0004,0


In [75]:
pred_cnt = tuple(submit["Y_LABEL"].value_counts())
a, b = pred_cnt
r = f"정상값 개수 : {a}, 이상값 개수 : {b}"
print(r)

정상값 개수 : 5308, 이상값 개수 : 733


In [76]:
submit.to_csv(f'./submits/submit{file_version}.csv', index=False)

In [77]:
record = {"ver": file_version, "CFG": CFG, "Val_result": choose_threshold_result, "pred_cnt": r}
record

{'ver': 35,
 'CFG': {'EPOCHS': 30,
  'LEARNING_RATE': 0.01,
  'BATCH_SIZE': 256,
  'SEED': 43,
  'log_transform_turn': 1,
  'skew_cut': 0.01,
  'vif_cut': 10,
  'corr_cut': 0.006,
  'outlier_corr_cut': 100,
  'T_Thresh': 0.35,
  'S_Thresh': 0.35,
  'reduct': 'sum',
  'drop_rate': 0.2},
 'Val_result': 'Best Threshold : [0.2], Score : [0.58209]',
 'pred_cnt': '정상값 개수 : 5308, 이상값 개수 : 733'}